In [ ]:
# This notebook shows how to perform AI incident classification using GPT-5-mini batch API calls.
# It demonstrates the prompts and workflow for labeling incidents with 32 predefined categories,
# including how to prepare the input file, submit the batch job and retrieve the output files.

### Imports
Load required libraries.

In [ ]:
import os
import json
import time
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

### OpenAI API Key Configuration  
Load the OpenAI API key from the .env file.

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

### Data Loading and Overview
Load the dataset containing AI incident mitigation texts and check its structure and summary information.  
Each row represents a single AI incident, and the "mitigation_taken" column lists all mitigation actions associated with that incident.

In [ ]:
df = pd.read_csv('DATASET.csv')

In [ ]:
# Filter the dataset to show rows where no mitigation actions were taken
df[df["mitigation_taken"] == "['No mitigation taken']"]

In [ ]:
# Remove rows where no mitigation actions were taken
df = df[df['mitigation_taken'] != '[\'No mitigation taken\']']

In [ ]:
df.info()

In [ ]:
df.head()

### OpenAI Batch API
This section prepares and configures batch API calls using GPT-5-mini to classify AI incident mitigation actions into 32 predefined subcategories.

- **Dataset:** We use a dataset of AI incident texts, where each row represents a single incident and the `mitigation_taken` column contains the mitigation actions associated with that incident. Each mitigation is assigned a unique `custom_id`.

- **What is being done:** The code splits the dataset into batches and constructs structured tasks for the OpenAI Batch API. Each task contains a set of mitigation statements and instructions for GPT-5-mini to assign one to maximum 5 subcategory labels to each mitigation.

- **How it is being done:**
  1. **Batching:** The dataset is divided into chunks (default 5 incidents per batch).
  2. **Data processing:** For each batch, mitigation statements are extracted and formatted with their associated IDs to create a clear input for classification.
  3. **Task creation:** For each batch, a task dictionary is created containing:
     - A unique `custom_id` for the batch
     - The API endpoint (`/v1/chat/completions`)
     - GPT-5-mini as the model
     - **System instructions** defining the classification rules, constraints, and strict output format
     - **User instructions** listing each mitigation statement with its ID and specifying the 32 allowed subcategories, along with rules for handling missing, unclear or out-of-scope actions
  4. **Task collection:** Each prepared task is appended to a `tasks` list, ready to be submitted to the OpenAI Batch API for classification.

In [ ]:
batch_size = 5
tasks = []

for start in range(0, len(df), batch_size):
    batch = df.iloc[start:start + batch_size]

    mitigations_block = []
    for index, row in batch.iterrows():
        mitigation_text = (
            str(row['mitigation_taken']).strip()
            if pd.notna(row['mitigation_taken'])
            else ""
        )

        new_id = row['custom_id']

        mitigations_block.append(
            f"- ID: {new_id}. Mitigation: {mitigation_text}"
        )

    task = {
        "custom_id": f"batch_{start}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-5-mini",
            "messages": [
                {
                    "role": "system",
                    "content":
                    "You are an AI incident classification expert.\n"
                    "Your task is to precisely classify AI incident mitigation actions into predefined mitigation subcategories.\n"
                    "You must follow the user instructions exactly.\n"
                    "You must not add explanations, reasoning, commentary, or extra text.\n"
                    "You must output ONLY valid, machine-parseable JSON.\n"
                    "You must not invent, infer, generalize, or rename categories.\n"
                    "All strings must use standard double quotes (\")."
                },
                {
                    "role": "user",
                    "content": (
                        "TASK:\n"
                        "You will be given a list of AI incident mitigation descriptions.\n"
                        "Each entry includes an ID and a mitigation description.\n"
                        "For EACH mitigation, assign one or more mitigation SUBCATEGORY labels.\n\n"

                        "DEFINITION:\n"
                        "A \"subcategory\" refers to a specific, fine-grained mitigation type.\n\n"

                        "ALLOWED SUBCATEGORIES (use ONLY these exact strings):\n"
                        "- \"Board Structure & Oversight\"\n"
                        "- \"Risk Management\"\n"
                        "- \"Conflict of Interest Protections\"\n"
                        "- \"Whistleblower Reporting & Protection\"\n"
                        "- \"Safety Decision Frameworks\"\n"
                        "- \"Environmental Impact Management\"\n"
                        "- \"Societal Impact Assessment\"\n"
                        "- \"Model & Infrastructure Security\"\n"
                        "- \"Model Alignment\"\n"
                        "- \"Model Safety Engineering\"\n"
                        "- \"Content Safety Controls\"\n"
                        "- \"Testing & Auditing\"\n"
                        "- \"Data Governance\"\n"
                        "- \"Access Management\"\n"
                        "- \"Staged Deployment\"\n"
                        "- \"Post-Deployment Monitoring\"\n"
                        "- \"Incident Response & Recovery\"\n"
                        "- \"Incident Investigation\"\n"
                        "- \"System Documentation\"\n"
                        "- \"Risk Disclosure\"\n"
                        "- \"Incident Reporting\"\n"
                        "- \"Governance Disclosure\"\n"
                        "- \"Third-Party System Access\"\n"
                        "- \"User Rights & Recourse\"\n"
                        "- \"Training & Supportive Measures\"\n"
                        "- \"System & Feature Restrictions\"\n"
                        "- \"Usage & Access Limitations\"\n"
                        "- \"Denial & Defensive Based Actions\"\n"
                        "- \"Court & Law Enforcement Interventions\"\n"
                        "- \"Regulatory, Policy & Legal Mandates\"\n"
                        "- \"Financial, Economic & Compensation Remedies\"\n"
                        "- \"Market Access & Commercial Restrictions\"\n"

                        "CLASSIFICATION RULES:\n"
                        "1. Assign all subcategories that clearly apply.\n"
                        "2. If the mitigation text explicitly states that no action was taken → return [\"None\"].\n"
                        "3. If an action is mentioned but does NOT match any allowed subcategory → return [\"Other\"].\n"
                        "4. Do NOT infer actions that are not explicitly stated.\n"
                        "5. Do NOT merge, generalize, or invent subcategories.\n"
                        "6. Do NOT explain your reasoning.\n"
                        "7. Do NOT return any text outside the required JSON structure.\n"
                        "8. Subcategory labels must match the allowed list EXACTLY (case-sensitive).\n"
                        "9. Each mitigation must be classified independently.\n"
                        "10. Assign a maximum of 5 subcategories per mitigation. "
                        "If more than 5 subcategories could apply, select the 5 most directly and explicitly supported by the mitigation text.\n\n"

                        "OUTPUT FORMAT (STRICT):\n"
                        "Return ONLY a single JSON object with this exact structure:\n\n"
                        "{\n"
                        "  \"results\": {\n"
                        "    \"<MITIGATION_ID>\": {\n"
                        "      \"mitigation_text\": \"<original mitigation text exactly as given>\",\n"
                        "      \"assigned_subcategories\": [\"<subcategory>\", \"...\"]\n"
                        "    }\n"
                        "  }\n"
                        "}\n\n"

                        "MITIGATIONS TO CLASSIFY:\n"
                        "<mitigations>\n"
                        + "\n".join(mitigations_block) +
                        "\n</mitigations>"
                    )
                }
            ]
        }
    }

    tasks.append(task)

In [ ]:
file_name = "TASK.jsonl"

# Write each task object to a JSONL file
with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [ ]:
# Upload the batch input file to OpenAI and register it for batch processing
batch_file = client.files.create(
    file=open(file_name, "rb"),
    purpose="batch"
)
print(batch_file)

In [ ]:
# Create a batch job using the uploaded input file
batch_job = client.batches.create(
    input_file_id=batch_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)

In [ ]:
# Optional - poll the batch job status until completion
while True:
    batch_job = client.batches.retrieve(batch_job.id)
    if batch_job.status != "completed":
        time.sleep(10)
        print(batch_job.status)
    else:
        print(f"job {batch_job.id} is done")
        break

In [ ]:
# Print batch job information
batch = client.batches.retrieve(batch_job.id)
print(batch)

In [ ]:
output_file_id = batch.output_file_id
print(output_file_id)

In [ ]:
# Save batch output file locally in JSONL format
with open("RESULT.jsonl", "wb") as f:
    for chunk in client.files.content(output_file_id).iter_bytes():
        f.write(chunk)